In [1]:
import os
import sys
import platform

import torch
import torch.nn
import torchvision.datasets


from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.utils import save_image

In [2]:
print("Python Version: {}".format(platform.python_version()))
print("Torch Version: {}".format(torch.__version__))

Python Version: 3.7.1
Torch Version: 0.4.1


# Get MNIST Data

In [3]:
num_epochs = 11
batch_size = 128
learning_rate = 1e-3

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

In [5]:
mnistTrainData = torchvision.datasets.MNIST("Data", download=True, train=True, transform=transform)
mnistTestData = torchvision.datasets.MNIST("Data", download=True, train=False, transform=transform)

In [6]:
print("Number of Training Samples: {}".format(len(mnistTrainData)))
print("Number of Testing Samples: {}".format(len(mnistTestData)))

Number of Training Samples: 60000
Number of Testing Samples: 10000


In [7]:
trainDataLoader = torch.utils.data.DataLoader(mnistTrainData, batch_size=batch_size, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(mnistTestData, batch_size=batch_size, shuffle=True)

# Autoencoder Model

In [8]:
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(torch.nn.Linear(28 * 28, 144))
        self.decoder = torch.nn.Sequential(torch.nn.Linear(144, 28 * 28), torch.nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Train Model

In [9]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [10]:
model = Autoencoder()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(trainDataLoader):
        img = data.view(data.size(0), -1)
        img = Variable(img)
        
        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, 'image_{}.png'.format(epoch))


epoch [1/11], loss:0.0477
epoch [2/11], loss:0.0314
epoch [3/11], loss:0.0250
epoch [4/11], loss:0.0221
epoch [5/11], loss:0.0198
epoch [6/11], loss:0.0187
epoch [7/11], loss:0.0175
epoch [8/11], loss:0.0157
epoch [9/11], loss:0.0156
epoch [10/11], loss:0.0147
epoch [11/11], loss:0.0161


# Visualize Ouput

In [47]:
x = model.encoder(mnistTrainData[0][0].view(-1))
x.cpu().data

tensor([-2.5972,  2.0335, -1.4035, -5.9272,  4.1230, -0.5463, -7.8617, -2.3058,
        -1.6839, -2.7362,  0.8151, -3.0568,  2.8634, -4.4560, -4.3327, -1.8651,
         2.2705, -2.7953,  0.4271, -2.2346,  2.1029,  0.3604, -2.3188, -2.2435,
        -1.1194,  2.6461,  4.2320, -2.2890, -1.7516, -3.6522,  2.1152,  2.1692,
         4.0917, -2.0081, -2.4519,  3.7258, -0.7955, -0.4441, -0.4540,  2.0830,
        -2.3765,  1.6434,  0.1619, -3.8213,  0.3543, -0.6025,  0.7139,  2.0157,
         3.0786,  2.5683, -6.6893, -2.4027, -2.4047, -2.6544,  2.7776,  0.4452,
         4.5585, -3.3856,  4.9795, -7.6465, -0.4484,  1.2471,  0.7143, -1.0763])

In [48]:
x = 0.5 * (x + 1)
x = x.clamp(0, 1)
x = x.view(1, 1, 12, 12)
save_image(x, 'test.png'.format(epoch))


RuntimeError: invalid argument 2: size '[1 x 1 x 12 x 12]' is invalid for input with 64 elements at /Users/soumith/miniconda2/conda-bld/pytorch_1532624435833/work/aten/src/TH/THStorage.cpp:84

# Visualize Latent Properties